<p style="font-family: Arial; font-size:2.5em;color:green;"> WIMTACH WORKSHOP </p>
<p style="font-family: Arial; font-size:2.5em;color:orange;"> Retail Analytics Using Python </p>

## Data: Superstore Data 

##### Import Libraries

In [ ]:
!pip install squarify

In [ ]:
# Import Libarires
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import squarify


###### Import Data

In [ ]:
data= pd.read_excel('superstore.xls')
data.head()

##### Shape of the Data

In [ ]:
data.shape

In [ ]:

print("Number of attrtibutes or variables",data.shape[1])
print("Number of customers",data.shape[0])

## List of Variables

In [ ]:
data.columns

### Descriptive  Statistics

In [ ]:
data.describe()

### Data Types

In [ ]:
data.info()

###  Missing Values In Percentage

In [ ]:
m=pd.DataFrame(((data.isnull().sum())/len(data))*100).reset_index()
m.columns=['Attributes','Percentage of Missing']
m

## Exttracting Month and Year from Ship Date

In [ ]:
data['Month']= pd.DatetimeIndex(data['Ship Date']).month
data['Year']= pd.DatetimeIndex(data['Ship Date']).year

In [ ]:
data.columns

In [ ]:
 data['Order Date'].dt.day_name()

### Exploring  Sales

#### Checking outliers

In [ ]:
dt_sales= data[['Ship Date','Sales']].set_index('Ship Date')
dt_sales.plot()

### Removing Outliers

In [ ]:
# outliers = data[data['SalesAmount'] > (data['SalesAmount'].mean() + (3 * data['SalesAmount'].std()))] 
# outliers2 = data[data['SalesAmount'] < (data['SalesAmount'].mean() - (3 * data['SalesAmount'].std()))]

#removing outliers from data:
print(data['Sales'].mean(), data['Sales'].std(), data['Sales'].mean() - 3 * data['Sales'].std(), data['Sales'].mean() + 3 * data['Sales'].std())
data = data[data['Sales'] < (data['Sales'].mean() + (3 * data['Sales'].std()))] 
data = data[data['Sales'] > (data['Sales'].mean() - (3 * data['Sales'].std()))]

In [ ]:
dt_sales2= data[['Ship Date','Sales']].set_index('Ship Date')

dt_sales2.plot()

### Year and Month  Wise Sales

In [ ]:
data.groupby(data.Year).sum()['Sales']

In [ ]:
data.groupby(data.Year).sum()['Sales'].plot(kind='bar')

In [ ]:
data.groupby(data.Month).sum()['Sales']

In [ ]:
data.groupby(data.Month).sum()['Sales'].plot(kind='bar')

### Product Category wise Sales in Percentage

In [ ]:
data.columns

In [ ]:
dk = data.groupby(['Category']).agg({'Sales':np.sum}).reset_index()
fig1 = plt.figure(figsize =(10, 7)) 
plt.pie(dk.Sales, labels = dk.Category,autopct='%1.3f%%')
# show plot 
plt.show() 



In [ ]:
# Category
di=dk.sort_values('Sales')
fig2 = plt.figure(figsize = (10, 5)) 
plt.bar(di.Category, di.Sales, color ='green', width = 0.4)
plt.xlabel("Categories") 
plt.ylabel("Total Sales")  
# show plot 
plt.show()

### Week Day Wise Sales 

In [ ]:
dj = data.groupby(['Order Date', 'Order ID']).size()
new_df = dj.to_frame(name = 'size').reset_index()
new_df['NumberofOrders']=1
new_df['day_of_week'] = new_df['Order Date'].dt.day_name()

dataSales = new_df.groupby('day_of_week')['NumberofOrders'].agg('sum').reset_index()
dataSales = dataSales.loc[[1,5,6,4,0,2,3], :]

# #plotting bar chart
fig = plt.figure(figsize = (15, 5)) 
plt.bar(dataSales.day_of_week, dataSales.NumberofOrders, color ='maroon', width = 0.6)
plt.xlabel("Day of week") 
plt.ylabel("Total orders") 
plt.show()

In [ ]:
new_df

In [ ]:
dataSales

## Count of Sales per Year

In [ ]:
data.groupby('Year')['Order ID'].count()

In [ ]:
data.groupby('Year')['Order ID'].count().plot(kind= 'bar')

# Region wise analysis

In [ ]:
data['Region'].unique()

In [ ]:
data.groupby('Region')['Sales'].sum().plot(kind='bar')

## Group by using Year and Category

In [ ]:
data.groupby(['Year','Category'])['Sales'].sum().reset_index()

## Pivot Table for Sales across Year and Category

In [ ]:
pd.pivot_table(data, values='Sales',index='Year',columns=['Category'],aggfunc= np.sum).reset_index()

In [ ]:
table= pd.pivot_table(data, values='Sales',index='Year',columns=['Category'],aggfunc= np.sum).reset_index()

In [ ]:
table.plot.bar(x='Year',y=['Furniture', 'Office Supplies','Technology'],stacked=True)

In [ ]:
table.plot.bar(x='Year',y=['Furniture', 'Office Supplies','Technology'])

In [ ]:
table.plot.bar(x='Year',y=['Furniture', 'Office Supplies','Technology'])
plt.xticks(rotation=0)
plt.xlabel('Year of the Businesss')
plt.ylabel("Sum of the sales amount")
plt.title(" The Sales Details")
          
plt.show()

# Month wise analysis

In [ ]:
data.groupby(['Month','Category'])['Sales'].sum().reset_index()

In [ ]:
data.groupby('Month')['Sales','Profit'].sum().plot(kind='bar')

## Month and Region Wise Exploration

In [ ]:
data.groupby([pd.Grouper(key='Ship Date', freq='M'), 'Region'])\
    .agg(unique_items=('Product ID', 'nunique'),
         total_Sales=('Sales','sum'),
         total_Profit=('Profit','sum')).head(20)

In [ ]:
data.resample('M',on='Ship Date').Sales.sum().head(10)

In [ ]:
data_M= data.resample('M',on='Order Date').Sales.sum()

In [ ]:
data_M= pd.DataFrame(data_M).reset_index()
data_M.head()

## Filtering

In [ ]:
dl= data.set_index('Order Date')
dl.loc['2016':'2017']

In [ ]:
data.loc[data['Region']== 'South',['Region','Sales'] ]

###  Interactive Graph  with Slide Bar using Plotly

In [ ]:
import plotly.express as px
fig = px.line(data_M, x = 'Order Date',y='Sales',title = 'Monthly Sales')

fig.update_xaxes(rangeslider_visible=True)
fig.show()   

# RFM Analysis

#### RFM stands for Recency, Frequency, and Monetary value, each corresponding to some key customer trait. These RFM metrics are important indicators of a customer’s behavior because frequency and monetary value affects a customer’s lifetime value, and recency affects retention, a measure of engagement.


#### Recency: The number of days between present date (i.e. Today) and date of last purchase of each customer. First define today's date and then get the `.max()` function of `InvoiceDate` to calculate the number of days between today and date of last purchase.

####  Frequency: How often has a customer transacted or interacted with the brand during a particular period of time? Clearly, customers with frequent activities are more engaged, and probably more loyal, than customers who rarely do so. And one-time-only customers are in a class of their own.

####  Monetary: Also referred to as “monetary value,” this factor reflects how much a customer has spent with the brand during a particular period of time. Big spenders should usually be treated differently than customers who spend little. Looking at monetary divided by frequency indicates the average purchase amount – an important secondary factor to consider when segmenting customers.

###  RFM factors illustrate these facts:
##### the more recent the purchase, the more responsive the customer is to promotions
##### the more frequently the customer buys, the more engaged and satisfied they are
##### monetary value differentiates heavy spenders from low-value purchasers   

### The maximum and minimum date 

In [ ]:
data['Order Date'].max(), data['Order Date'].min()

In [ ]:

TODAY = dt.datetime(2017,12,31)

In [ ]:
rfm = data.groupby("Customer ID").agg({"Order Date": lambda date: (TODAY - date.max()).days,
                                        "Order ID": lambda num: num.nunique(),
                                        "Sales": lambda price: price.sum()})

In [ ]:
# Change the name of columns
rfm.columns=["recency", "frequency", "monetary"]

In [ ]:
rfm

In [ ]:
print("Recency: {} to {}".format(rfm.recency.min(), rfm.recency.max()))
print("Frequency: {} to {}".format(rfm.frequency.min(), rfm.frequency.max()))
print("Monetary: {} to {}".format(rfm.monetary.min(), rfm.monetary.max()))

# Customer Life Time Value

####  Customer Lifetime Value is a monetary value that represents the amount of revenue or profit a customer will give the company over the period of the relationship" (Source). CLTV demonstrates the implications of acquiring long-term customers compare to short-term customers. Customer lifetime value (CLV) can help you to answers the most important questions about sales to every company: 


    How to Identify the most profitable customers?
    How can a company offer the best product and make the most money?
    How to segment profitable customers?
    How much budget need to spend to acquire customers?


# Aggregate Model

The most simplest and the oldest method of computing CLV is this Aggregate/Average method. This assumes a constant average spend and churn rate for all the customers.

This method does not differentiate between customers and produces a single value for CLV at an overall Level. This leads to unrealistic estimates if some of the customers transacted in high value and high volume, which ultimately skews the average CLV value.

The General Formula for calculating CLV is:


### CLV = ((Average Sales X Purchase Frequency) / Churn) X Profit Margin  

**Where,**  
  **Average Sales = TotalSales/Total no. of orders**
   
   **Purchase Frequency = Total no. of orders/Total unique customers**
   
   **Retention rate = Total no. of orders greater than 1/ Total unique customers**
   
   **Churn = 1 - Retention rate**
   
   **Profit Margin = Based on business context**

In this example case,  the Profit margin for each transaction is assumed to be 5%.

### Slicing Data for CLV Analaysis

In [ ]:
data_clv= data[['Customer ID','Order ID', 'Order Date', 'Sales']]
data_clv.head(2)

In [ ]:

customer = data_clv.groupby('Customer ID').agg({'Order Date':lambda x: (x.max() - x.min()).days, 
                                                   'Order ID': lambda x: len(x),
                                                  'Sales': lambda x: sum(x)})

customer.columns = ['Days', 'Frequency', 'TotalSales']
customer.head()

In [ ]:
customer.shape

In [ ]:
data_clv['Order Date'].max()

In [ ]:
data_clv['Order Date'].min()

In [ ]:
# Calculating the necessary variables for CLV calculation
Average_sales = round(np.mean(customer['TotalSales']),2)
print(f"Average sales: ${Average_sales}")

Purchase_freq = round(np.mean(customer['Frequency']), 2)
print(f"Purchase Frequency: {Purchase_freq}")

Retention_rate = customer[customer['Frequency']>1].shape[0]/customer.shape[0]
churn = round(1 - Retention_rate, 2)
print(f"Churn: {churn}%")

In [ ]:
customer['TotalSales'].mean()

In [ ]:
# Calculating the CLV
Profit_margin = 0.05 

CLV = round(((Average_sales * Purchase_freq/churn)) * Profit_margin, 2)
print(f"The Customer Lifetime Value (CLV) for each customer is: ${CLV}")

#   Conversion Rate

In [ ]:
data.groupby('Year')['Order ID'].count()

In [ ]:
dt= pd.DataFrame(data.groupby('Year')['Order ID'].count()).reset_index()
dt.columns=['Year', 'No. of Transactions']
dt

In [ ]:
dt['Footfall']= [10000,15000,20000,25000,30000]

In [ ]:
dt

In [ ]:
dt['Conversion Rate']= dt['No. of Transactions']/dt['Footfall']*100
dt

# Market Basket Analysis

# Frequency of Most Popular Item 

In [ ]:
# looking at the frequency of most popular items 

plt.rcParams['figure.figsize'] = (18, 7)
color = plt.cm.copper(np.linspace(0, 1, 20))
data['Product Name'].value_counts().head(40).plot.bar(color = color)
plt.title('frequency of most popular items', fontsize = 20)
plt.xticks(rotation = 90 )
plt.grid()
plt.show()

In [ ]:
y = data['Product Name'].value_counts().head(50).to_frame()


In [ ]:
# plotting 

plt.rcParams['figure.figsize'] = (20, 20)
color = plt.cm.cool(np.linspace(0, 1, 50))
squarify.plot(sizes = y.values, label = y.index, alpha=.8, color = color)
plt.title('Tree Map for Popular Items')
plt.axis('off')
plt.show()

In [ ]:


from wordcloud import WordCloud

plt.rcParams['figure.figsize'] = (15, 15)
wordcloud = WordCloud(background_color = 'white', width = 1200,  height = 1200, max_words = 121).generate(str(data['Product Name']))
plt.imshow(wordcloud)
plt.axis('off')
plt.title('Most Popular Items',fontsize = 20)
plt.show()

In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
data['Product Name'].value_counts().head(10)

In [ ]:
df=data.groupby("Product Name").filter(lambda x: len(x) > 14)
df.head()

In [ ]:
df['Product Name'].value_counts()

In [ ]:
dt= df.groupby(['Order ID','Product Name'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('Order ID')
dt

In [ ]:
def one_hot_encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

dt_encode= dt.applymap(one_hot_encode)

In [ ]:
dt_efilter= dt_encode[(dt_encode >0).sum(axis=1)>=2]

In [ ]:
dt_efilter

In [ ]:
frequent_itemsets_plus= apriori(dt_efilter, min_support= 0.05,use_colnames= True).sort_values('support', ascending= False).reset_index(drop=True)

In [ ]:
rules = association_rules(frequent_itemsets_plus, metric="lift", min_threshold=2)
rules